In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math
import pandas as pd

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [8]:
def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2 + (point1.z - point2.z)**2)

In [20]:
cap = cv2.VideoCapture('DATA_SAMPLES/DATA8.mp4')
# cap = cv2.VideoCapture(0)
data = []

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertendo para RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for num, hand_landmarks in enumerate(results.multi_hand_landmarks):
                hand_label = results.multi_handedness[num].classification[0].label  # 'Left' ou 'Right'

                # Pegando os pontos das pontas dos dedos
                thumb_tip = hand_landmarks.landmark[4]
                index_tip = hand_landmarks.landmark[8]
                middle_tip = hand_landmarks.landmark[12]
                ring_tip = hand_landmarks.landmark[16]
                pinky_tip = hand_landmarks.landmark[20]
                wrist = hand_landmarks.landmark[0]

                # Pegando as bases dos dedos
                index_base = hand_landmarks.landmark[5]
                middle_base = hand_landmarks.landmark[9]
                ring_base = hand_landmarks.landmark[13]
                pinky_base = hand_landmarks.landmark[17]

                # Calculando distâncias entre os dedos
                thumb_index_dist = calculate_distance(thumb_tip, index_tip)
                index_middle_dist = calculate_distance(index_tip, middle_tip)
                middle_ring_dist = calculate_distance(middle_tip, ring_tip)
                ring_pinky_dist = calculate_distance(ring_tip, pinky_tip)
                palm_width = calculate_distance(index_base, pinky_base)
                palm_height = calculate_distance(middle_base, wrist)
                thumb_to_wrist = calculate_distance(thumb_tip, wrist)
                index_to_wrist = calculate_distance(index_tip, wrist)
                middle_to_wrist = calculate_distance(middle_tip, wrist)

                # Definição dos gestos com base nas distâncias
                if thumb_to_wrist > 0.35 and index_to_wrist > 0.35 and middle_to_wrist > 0.35:
                    gesture = 0  # Mão aberta espalmada
                
                elif (
                    thumb_index_dist < 0.2 and
                    index_middle_dist < 0.2 and
                    middle_ring_dist < 0.2 and
                    ring_pinky_dist < 0.2
                ):
                    gesture = 1  # Mão em formato de garra
                
                else:
                    gesture = 5  # Gesto desconhecido

                # Exibir texto do gesto detectado
                cv2.putText(image, f"{hand_label}: {gesture}", (10, 50 + num * 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Desenhar os pontos da mão
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Coleta das coordenadas e rótulo
                landmarks = [landmark for landmark in hand_landmarks.landmark]
                flattened = [coord for landmark in landmarks for coord in (landmark.x, landmark.y, landmark.z)]

                data.append(flattened + [gesture])

        cv2.imshow('Detecção de Gestos', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [22]:
##Salvar dados em arquivo CSV
#df = pd.DataFrame(data, columns=[f'landmark_{i}_x' for i in range(21)] + [f'landmark_{i}_y' for i in range(21)] + [f'landmark_{i}_z' for i in range(21)] + ['gesture'])
#df.to_csv('Exerc3.csv', index=False)